In [3]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [4]:
from otgnn.models import GCN, compute_ot
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.mol_opt import MolOpt

from rdkit.Chem import MolFromSmiles

import torch
from torch.autograd import Variable
import numpy as np
import time

In [6]:
data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, False)

In [7]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [8]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"
molopt = MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
)

In [12]:
molopt.ref

tensor([[ 0.1016,  0.9755,  0.4594,  ...,  0.7711,  0.0087, -0.1619],
        [ 0.9830,  0.9040,  0.4739,  ...,  0.3547,  0.1629,  0.8527],
        [-0.4082,  0.0397, -1.5386,  ...,  1.9711, -0.4085,  0.9072],
        ...,
        [ 0.8164, -0.2478,  0.1603,  ..., -0.1824, -0.9644,  0.2617],
        [ 2.2706, -1.9566,  0.4781,  ...,  0.4993,  0.3142, -0.8587],
        [-0.7495, -1.9705,  0.6760,  ...,  2.6502,  0.3984,  0.0431]],
       device='cuda:0', requires_grad=True)

In [13]:
molopt.ref

tensor([[ 0.1016,  0.9755,  0.4594,  ...,  0.7711,  0.0087, -0.1619],
        [ 0.9830,  0.9040,  0.4739,  ...,  0.3547,  0.1629,  0.8527],
        [-0.4082,  0.0397, -1.5386,  ...,  1.9711, -0.4085,  0.9072],
        ...,
        [ 0.8164, -0.2478,  0.1603,  ..., -0.1824, -0.9644,  0.2617],
        [ 2.2706, -1.9566,  0.4781,  ...,  0.4993,  0.3142, -0.8587],
        [-0.7495, -1.9705,  0.6760,  ...,  2.6502,  0.3984,  0.0431]],
       device='cuda:0', requires_grad=True)

In [14]:
molopt.Nref = molopt.ref.shape[0]

In [15]:
tg_embedding = torch.empty(molopt.Nref * len(X.scope), 50, device = args.device)

In [16]:
tg_embedding.shape

torch.Size([1920, 50])

In [17]:
embedding = molopt.encode(X)

/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/ot/lp/__init__.py:276: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


In [28]:
for i, (stx, lex) in enumerate(X.scope):
    narrow = embedding.narrow(0, stx, lex)
    H = np.ones(lex)/lex
    _,_,OT_xy,_ = compute_ot(narrow, molopt.ref, H, molopt.Href, sinkhorn_entropy = 0.1, device = args.device,
                            opt_method = 'emd')
    V = torch.matmul((molopt.Nref * OT_xy).T, narrow) - molopt.ref
    tg_embedding[i*molopt.Nref : (i+1)*molopt.Nref,:] = V / np.sqrt(molopt.Nref)

In [29]:
V = torch.matmul((molopt.Nref * OT_xy).T, narrow) - molopt.ref

In [30]:
tg_embedding

tensor([[-2.9584e-03, -1.4681e-01, -9.3173e-02,  ..., -1.0403e-01,
         -1.1898e-02,  3.8689e-02],
        [-1.3515e-01, -1.6102e-01, -7.7645e-02,  ..., -5.3740e-02,
         -1.2072e-02, -1.4962e-01],
        [ 9.9245e-02,  3.4363e-03,  2.2038e-01,  ..., -2.8568e-01,
          7.4675e-02, -1.3289e-01],
        ...,
        [-1.5738e-01,  1.6620e-02, -5.6824e-02,  ...,  3.0537e-02,
          9.0969e-02, -3.2934e-02],
        [-3.0833e-01,  2.6196e-01, -8.2754e-02,  ..., -7.3652e-02,
         -1.1134e-02,  9.4832e-02],
        [ 1.0359e-01,  2.9045e-01, -6.9150e-02,  ..., -4.2287e-01,
         -4.6867e-02,  1.4428e-04]], device='cuda:0', grad_fn=<CopySlices>)

In [32]:
molopt.ref

tensor([[ 0.1016,  0.9755,  0.4594,  ...,  0.7711,  0.0087, -0.1619],
        [ 0.9830,  0.9040,  0.4739,  ...,  0.3547,  0.1629,  0.8527],
        [-0.4082,  0.0397, -1.5386,  ...,  1.9711, -0.4085,  0.9072],
        ...,
        [ 0.8164, -0.2478,  0.1603,  ..., -0.1824, -0.9644,  0.2617],
        [ 2.2706, -1.9566,  0.4781,  ...,  0.4993,  0.3142, -0.8587],
        [-0.7495, -1.9705,  0.6760,  ...,  2.6502,  0.3984,  0.0431]],
       device='cuda:0', requires_grad=True)

In [33]:
V

tensor([[-1.9532e-01, -8.9288e-01, -5.1413e-01,  ..., -7.9166e-01,
          7.2271e-02,  1.5796e-01],
        [-9.8297e-01, -9.0404e-01, -4.7385e-01,  ..., -3.5465e-01,
         -1.6292e-01, -8.5272e-01],
        [ 6.1862e-01,  2.8528e-02,  1.3885e+00,  ..., -1.7882e+00,
          5.0211e-01, -8.5508e-01],
        ...,
        [-9.9534e-01,  1.0512e-01, -3.5939e-01,  ...,  1.9313e-01,
          5.7534e-01, -2.0830e-01],
        [-1.9500e+00,  1.6568e+00, -5.2338e-01,  ..., -4.6582e-01,
         -7.0417e-02,  5.9977e-01],
        [ 6.5516e-01,  1.8370e+00, -4.3734e-01,  ..., -2.6745e+00,
         -2.9641e-01,  9.1248e-04]], device='cuda:0', grad_fn=<SubBackward0>)